### This notebook contains the code for RAG implementation with Qdrant DB for Proposal Pro backend
Steps followed:
- Upload files
- for file in files:
    - for each page in file: 
        - update metadata
        - chunkify with metadata (RecursiveCharacterTextSplitter)
        - append to a docs_list
    - return docs_list
    - update metadata for each chunk in docs_list:
        - chunk.metadata.update({"tags": "", "flename": ""})
    - get embedding model to use
    - create embeddings for each chunk (OR):
    - create/update vector DB with the docs_list, embedding model, using collection_name (creates embeddings too)


## DB Architecture:
Each collection in the Qdrant DB will be associated with a ProjectID.
The collections will be simple, Vector databases with: -

- **collection_name**: *ProjectName_ProjectID*.
- **VectorParams**: Size=1024, Distance=COSINE
- **PointStruct**: id, vector=chunk_embedding, payload=chunk_metadata

A Qdrant collection can be viewed as a pool of embeddings from all the docs associated with a Project, with the chunk info (Filename, Page No.) maintained in the payload.
For Retrieval, query is sent to a Qdrant collection which returns the Vectors?? or Docs?? for the
